Setup Spark

In [107]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar -xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
import time
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

spark

Tempo totale dal blocco sottostante: 11s

In [108]:
df = spark.read.csv('sample_data/exercise_2.txt',inferSchema=True, header =True)
df.show(5)

+---+-----+
| ID|Value|
+---+-----+
|  A|   -8|
|  Z|    6|
|  P|    5|
|  A|  -10|
|  X|    0|
+---+-----+
only showing top 5 rows



Calcolo la media dell'intera colonna Value

In [109]:
from pyspark.sql.functions import avg,col
total_avg = df.select(avg("Value")).collect()[0][0]
total_avg

-0.4998024

In ordine si realizzano le seguenti operazioni:

- GroupBy in base all'ID aggregando la colonna Value come media dei valori.
- Trasformazione tramite la funzione withColumn, realizzando la sottrazione del valore della media dell'ID meno la media totale.
- Sorting dei risultati in base all'ID.

In [110]:
gruppi = df.groupBy("ID").agg(avg("Value").alias("mean")).withColumn("Distance", col("mean")-total_avg).orderBy("ID").show(truncate=False)

+---+--------------------+---------------------+
|ID |mean                |Distance             |
+---+--------------------+---------------------+
|A  |-0.48894494270054806|0.010857457299451923 |
|B  |-0.5040795751005602 |-0.004277175100560249|
|C  |-0.4802683339846294 |0.019534066015370555 |
|D  |-0.5069762122598354 |-0.007173812259835377|
|E  |-0.49876027056187233|0.001042129438127648 |
|F  |-0.5044120398278836 |-0.004609639827883649|
|G  |-0.4942143719224343 |0.005588028077565699 |
|H  |-0.49937647582574646|4.2592417425352247E-4|
|I  |-0.5005939851191097 |-7.915851191097056E-4|
|J  |-0.4952244258872651 |0.004577974112734862 |
|K  |-0.5110657700235265 |-0.011263370023526509|
|L  |-0.5160278854083998 |-0.016225485408399853|
|M  |-0.5061216100901201 |-0.006319210090120153|
|N  |-0.5098104958920416 |-0.010008095892041657|
|O  |-0.49119270059338854|0.008609699406611437 |
|P  |-0.48528618379043337|0.014516216209566613 |
|Q  |-0.5034903605752987 |-0.003687960575298743|
|R  |-0.488324720102